In [1]:
import numpy as np

In [2]:
# Encapsulates a vector of each arm
class Arms:
    # Creates a vector of N arms
    def __init__(self, N):
      self.N = N
      self.reward_weights = np.random.rand(self.N)
      self.bid_weights = np.random.rand(self.N)
      self.cost_weights = self.bid_weights * np.random.rand(self.N)

      print("Average reward by arm:")
      print(self.reward_weights / 2, "\n")

      print("Average bid by arm:")
      print(self.bid_weights / 2, "\n")

      print("Average cost by arm:")
      print(self.cost_weights / 2, "\n")

    # Returns an np array of the reward for each arm
    def generate_rewards(self):
        return np.random.rand(self.N) * self.reward_weights

    # Returns an np array of the bid for each arm
    def generate_bids(self):
        return np.random.rand(self.N) * self.bid_weights

    # Returns an np array of the cost for each arm
    def generate_costs(self):
        return np.random.rand(self.N) * self.cost_weights

In [3]:
# Takes the average rewards vector, total visits vector, number of arms chosen per round, and current round
# and returns a vector of the upper confidence bounds
def compute_ucb_rewards(expected_rewards, visits, K, t):
    ucf = ((K + 1) * np.log(t) / visits) ** 1/2
    ucb = expected_rewards + ucf
    return ucb

In [4]:
# Performs one round of selecting every arm to initialize the expected_rewards, paying the maximum cost 
def first_round(arms, N):
    visits = np.full(N, 1)
    costs = arms.generate_costs()
    cost = sum(costs)
    expected_rewards = arms.generate_rewards()
    return visits, expected_rewards, cost

In [5]:
# Holds an auction, choosing the best K arms to explore depending on their bids during the action and
# depending on expected_rewards, visits, and the current round. Returns the ucb rewards of each arm, a new_visits
# one hot encoded vector for whether or not each arm is chosen, and the bids of each arm
def auction_results(arms, expected_rewards, visits, K, t):
    bids = arms.generate_bids()
    ucb_rewards = compute_ucb_rewards(expected_rewards, visits, K, t)
    score_to_index = sorted([(ucb_rewards[i], i) for i in range(len(ucb_rewards))], reverse=True)
    new_visits = np.zeros(len(ucb_rewards))
    for i in range(K):
        new_visits[score_to_index[i][1]] = 1
    return ucb_rewards, new_visits, bids

In [6]:
# Calculates the cost of this round
def compute_cost(arms, ucb_rewards, new_visits, bids):
    costs = arms.generate_costs()

    # Zero out unchosen arms
    chosen_costs = costs * new_visits
    chosen_ucb = ucb_rewards * new_visits
    chosen_bids = bids * new_visits

    # TODO: cost = sum of function of costs, chosen_ucb, chosen_bids
    cost = sum(chosen_costs) # for now
    return cost

In [7]:
# Returns a vector of the rewards of the chosen arms and 0 for unchosen arms
def get_rewards(arms, new_visits):
    rewards = arms.generate_rewards()
    
    # Zero out unchosen arms
    return rewards * new_visits

In [8]:
# Updates the average reward of each arm using the most recent rewards
def update_expectations(expected_rewards, new_rewards, visits):
    # If new_rewards[i] is zeroed out, replace the zero with the average, so it doesn't change the average
    new_rewards = np.array([new_rewards[i] if new_rewards[i] != 0
                                           else expected_rewards[i] for i in range(len(new_rewards))])
    
    expected_rewards = ((expected_rewards * visits) + new_rewards) / (visits + 1)
    return expected_rewards

In [9]:
# Performs one round. Returns vectors of the visits from that round, rewards for that round,
# and the float cost of the round
def round(arms, expected_rewards, visits, K, t):
    ucb_rewards, new_visits, bids = auction_results(arms, expected_rewards, visits, K, t)
    new_rewards = get_rewards(arms, new_visits)
    new_cost = compute_cost(arms, ucb_rewards, new_visits, bids)
    return new_visits, new_rewards, new_cost

In [10]:
# Runs the simulation
def AUCB(arms, N, K, B):
    total_reward = 0
    visits, expected_rewards, cost = first_round(arms, N)
    t = 1
    while True:
        new_visits, new_rewards, cost = round(arms, expected_rewards, visits, K, t)

        if B < cost:
            return total_reward, expected_rewards

        expected_rewards = update_expectations(expected_rewards, new_rewards, visits)
        visits = visits + new_visits
        total_reward = total_reward + new_rewards
        B = B - cost
        t = t + 1

In [11]:
N = 15  # Number of arms
K = 5  # Number of arms to choose per round
B = 100000  # Budget
arms = Arms(N)
reward_by_arm, expected_rewards = AUCB(arms, N, K, B)
total_reward = sum(reward_by_arm)

print("Total reward:")
print(total_reward, "\n")

print("Total reward gained from each arm:")
print(reward_by_arm, "\n")

print("Expected reward of each arm:")
print(expected_rewards)

Average reward by arm:
[0.22339794 0.41333605 0.07814467 0.48771492 0.33717814 0.01176743
 0.00871015 0.18806528 0.02049152 0.32974141 0.28806893 0.32826848
 0.49754035 0.19386927 0.45464732] 

Average bid by arm:
[0.21663582 0.11407697 0.37550115 0.24880939 0.14929152 0.47228617
 0.31664349 0.47192671 0.39071824 0.33122177 0.07302861 0.42642641
 0.49533792 0.2507302  0.06799388] 

Average cost by arm:
[0.05740457 0.07509048 0.27440927 0.20800077 0.03220073 0.44357384
 0.2252081  0.0911773  0.28105664 0.14736739 0.0092561  0.3924213
 0.39499456 0.12716581 0.0384161 ] 

Total reward:
287123.79747900134 

Total reward gained from each arm:
[7.16802075e+01 5.41733809e+04 1.09014930e+01 6.41069401e+04
 4.07011190e+04 1.29987779e+00 9.10693637e-01 4.00435632e+01
 2.15264506e+00 1.85727323e+03 1.58354645e+02 1.05675728e+03
 6.52634488e+04 4.51238775e+01 5.96344111e+04] 

Expected reward of each arm:
[0.22557544 0.41372037 0.07999895 0.4892798  0.3368688  0.01213171
 0.00853093 0.17881201 0.0